In [13]:
from git import Repo
import datetime

### Clone step

In [2]:
git_url = 'git@github.com:JetBrains/kotlin.git'
repo_dir = './CloneDirectory'
Repo.clone_from(git_url, repo_dir)

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v git@github.com:JetBrains/kotlin.git ./CloneDirectory
  stderr: 'fatal: destination path './CloneDirectory' already exists and is not an empty directory.
'

### Fetch data step

In [3]:
kotlin_repo_path = './CloneDirectory/kotlin'
kotlin_repo = Repo(kotlin_repo_path)
commits = list(kotlin_repo.iter_commits('master', max_count=5000))

In [4]:
import csv
import pandas

row = ['Date', 'Author', 'Branch', 'Message', 'DayOfWeek', 'Hour', 'Day', 'Month', 'Year', 'Minute', 'Second']
commitData = pandas.DataFrame(columns=row)
commitData['Minute'] = commitData['Minute'].astype(int)

In [5]:
for commit in commits:
    commitDate = datetime.utcfromtimestamp(commit.authored_date) 
    h = commitDate.hour + commitDate.minute / 60. + commitDate.second / 3600.
    commitData = commitData.append({'Date': commitDate, 'Author': commit.author, 
                       'Branch': 'master', 'Message': commit.message, 
                       'DayOfWeek': commitDate.weekday(), 'Hour' : int(h), "Minute" : int(commitDate.minute), 
                                    "Second":  commitDate.second, "Month": commitDate.month, "Year": commitDate.year, 
                                    "Day": commitDate.day,
                                   },  ignore_index=True)
    

In [54]:
commitData = commitData.set_index('Date')
commitData.head()

,Author,Branch,Message,DayOfWeek,Hour,Day,Month,Year,Minute,Second
Date,,,,,,,,,,
2019-01-16 04:07:27,Vyacheslav Gerasimov,master,Build: implement useBootstrapStdlib flag\n\nEx...,2,4,16,1,2019,7,27
2019-01-16 04:01:20,Vyacheslav Gerasimov,master,Update bootstrap to 1.3.30-dev-991\n,2,4,16,1,2019,1,20
2019-01-15 16:19:50,Alexander Udalov,master,Extract JVM IR backend module out of compiler/...,1,16,15,1,2019,19,50
2019-01-21 14:47:04,Yan Zhulanow,master,Remove Android Extensions from the default JPS...,0,14,21,1,2019,47,4
2019-01-21 15:05:52,Vyacheslav Gerasimov,master,as34: Set until-build to 183.* to support AS 3...,0,15,21,1,2019,5,52


### Process data

In [62]:
#groupByHour = commitData.groupby(["Year", "Month", "Day", "Hour"])["Minute"].count()
groupByHour = commitData.resample('H')["Minute"].count()

In [63]:
groupByHour.head()

Date
2016-10-09 09:00:00    1
2016-10-09 10:00:00    0
2016-10-09 11:00:00    0
2016-10-09 12:00:00    0
2016-10-09 13:00:00    0
Freq: H, Name: Minute, dtype: int64

In [64]:
row = ['Date', 'Commits']
timeSeries = pandas.DataFrame(columns=row)
for index, value in groupByHour.items():
    timeSeries = timeSeries.append({'Date': datetime.datetime(index[0], index[1], index[2], index[3]), 
                                   'Commits': value},  ignore_index=True)

TypeError: 'Timestamp' object does not support indexing

In [49]:
timeSeries.head()

,Date,Commits
0,2016-10-09 09:00:00,1
1,2017-05-02 10:00:00,1
2,2017-07-25 18:00:00,1
3,2017-08-02 18:00:00,1
4,2017-08-14 11:00:00,1


### Write data

In [66]:
groupByHour.to_csv('commit_data.csv')